Pasos: 
1. Escribir el documento
2. Llenar el documento
3. Exportarlo a pdf
4. setear la contraseña del pdf
5. configurar el envío de correos

In [7]:
#Escribir y llenar el documento
import pandas as pd
from __future__ import print_function
#from mailmerge import MailMerge
import mailmerge
from datetime import date

#Exportarlo a pdf
from fpdf import FPDF 
from docx2pdf import convert

#Setear la contraseña
import PyPDF2
import os
import argparse

#Enviar correos masivos

Escribir y llenar el documento

In [15]:
def convert_to_pdf(input_docx, out_folder):
    from subprocess import  Popen
    LIBRE_OFFICE = r"/usr/bin/soffice"

    p = Popen([LIBRE_OFFICE, '--headless', '--convert-to', 'pdf', '--outdir',
               out_folder, input_docx])
    print([LIBRE_OFFICE, '--convert-to', 'pdf', input_docx])
    p.communicate()
    
    
def write_docs(cedula, nombre_completo, nombre_output,  template, path_output_docx = '', path_output_pdf = ''):
    """
    Esta función crea un .docx, lo pasa a formato pdf y setea un password. 
    Los fields del template son {'CEDULA', 'DURACIÓN', 'NOMBRE_COMPLETO', 'firma'}.
    Luego lo conviLIBRE_OFFICE = r"C:\Program Files\LibreOffice\program\soffice.exe"
    """

    document = mailmerge.MailMerge(template)
    print("Fields included in {}: {}".format(template,
                                             document.get_merge_fields()))

    document.merge(
        CEDULA = cedula,
        NOMBRE_COMPLETO = nombre_completo,
        DURACIÓN = 'Un mes',
        firma = '',
        VIGENCIA = '1 al 31 de diciembre',
    )
    #Crear documentos en word
    output_docx = path_output_docx + nombre_output + '.docx'
    
    document.write(output_docx)
    print('El documento .docx de '+ cedula + '-' + nombre_completo +'ha sido creado exitosamente.')
    
    input_docx = output_docx
    #Crear documentos en pdf
    convert_to_pdf(input_docx, path_output_pdf)


#documentation https://pbpython.com/python-word-template.html
# Define the templates - assumes they are in the same directory as the code


In [16]:
path = '01_codificadores/'
contrato = path + '01_CC6193-01 Minuta codificador(a) Icfes VF 231120.docx'
template = contrato
document = mailmerge.MailMerge(template)
path_output_docx = '02_pruebas/'
path_output_pdf = path_output_docx
#print("Fields included in {}: {}".format(template,
#                                             document.get_merge_fields()))

cedula = '1020738980'
nombre_completo = 'Esperanza vuitrago'
nombre_output = cedula + ' - ' + nombre_completo + ': Contrato como codificador. CNC'


write_docs(cedula, nombre_completo, nombre_output,  template, path_output_docx, path_output_pdf)


Fields included in 01_codificadores/01_CC6193-01 Minuta codificador(a) Icfes VF 231120.docx: {'CEDULA', 'NOMBRE_COMPLETO', 'VIGENCIA', 'DURACIÓN', 'firma'}
El documento .docx de 1020738980-Esperanza vuitragoha sido creado exitosamente.
['/usr/bin/soffice', '--convert-to', 'pdf', '02_pruebas/1020738980 - Esperanza vuitrago: Contrato como codificador. CNC.docx']


Exportarlo a .pdf y setear la contrase;a

In [17]:
#Set a password
def set_password(input_file, user_pass, owner_pass):
    """
    Function creates new temporary pdf file with same content,
    assigns given password to pdf and rename it with original file.
    """
    # temporary output file with name same as input file but prepended
    # by "temp_", inside same direcory as input file.
    path, filename = os.path.split(input_file)
    output_file = os.path.join(path, "contrato_" + filename)

 

    output = PyPDF2.PdfFileWriter()

 

    input_stream = PyPDF2.PdfFileReader(open(input_file, "rb"))

 

    for i in range(0, input_stream.getNumPages()):
        output.addPage(input_stream.getPage(i))

 

    outputStream = open(output_file, "wb")

 

    # Set user and owner password to pdf file
    output.encrypt(user_pass, owner_pass, use_128bit=True)
    output.write(outputStream)
    outputStream.close()

 

    # Rename temporary output file with original filename, this
    # will automatically delete temporary file
    #os.rename(output_file, input_file)

# def main():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-i', '--input_pdf', required=True,
#                         help='Input pdf file')
#     parser.add_argument('-p', '--user_password', required=True,
#                         help='output CSV file')
#     parser.add_argument('-o', '--owner_password', default=None,
#                         help='Owner Password')
#     args = parser.parse_args()
#     set_password(args.input_pdf, args.user_password, args.owner_password)



# if __name__ == "__main__":
#     main()
    


In [ ]:
input_file = path_output_pdf + 'contrato_' + nombre_output + '.pdf'
user_pass = cedula
owner_pass = '75369512480'
set_password(input_file, user_pass, owner_pass)

In [19]:
content = open('email_codificadores.txt')

#email_recipient = 'ebuitrago@cnccol.com'
email_subject = 'ebuitragod@gmail.com'
email_message = content.read()
attachment_location = input_file

send_email(email_recipient,
               email_subject,
               email_message,
               attachment_location)


NameError: name 'send_email' is not defined

In [35]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os.path
from docx2pdf import convert
import PyPDF2
import os
import argparse
import pandas as pd
from __future__ import print_function
from mailmerge import MailMerge
from datetime import date

correo_envio = 'recursoshumanos1@cnccol.com'
pass_correo_envio = 'Juanjo0921'

def send_email(email_recipient,
               email_subject,
               email_message,
               attachment_location = '',
               attachment_location1 = ''):

    email_sender = 'recursoshumanos1@cnccol.com'

    msg = MIMEMultipart()
    msg['From'] = email_sender
    msg['To'] = email_recipient
    msg['Subject'] = email_subject

    msg.attach(MIMEText(email_message, 'plain'))

    if attachment_location != '':
        filename = os.path.basename(attachment_location)
        attachment = open(attachment_location, "rb")
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                        "attachment; filename= %s" % filename)
        msg.attach(part)

        filename1 = os.path.basename(attachment_location1)
        attachment1 = open(attachment_location1, "rb")
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment1.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                        "attachment; filename= %s" % filename1)
        msg.attach(part)

    try:
        server = smtplib.SMTP('smtp.office365.com', 587)
        server.ehlo()
        server.starttls()
        server.login('recursoshumanos1@cnccol.com', 'Juanjo0921')
        text = msg.as_string()
        server.sendmail(email_sender, email_recipient, text)
        print(email_recipient)
        server.quit()
    except:
        print("SMPT server connection error")
    return True


In [25]:
content = open('email_codificadores.txt')

email_recipient = 'ebuitrago@cnccol.com'
email_subject = 'ebuitragod@gmail.com'
email_message = content.read()
attachment_location = input_file
attachment_location1 = '02_pruebas/1020738989 - Esperanza Buitrago: Contrato como codificador. CNC.pdf'

x = send_email(email_recipient,
               email_subject,
               email_message,
               attachment_location,
               attachment_location1)

email sent


In [27]:
email_message
type(x)

bool

In [28]:
#Convertir Monitores.xlsx en diccionario
#===paths
path = '01_codificadores/'
path_contratos = path + '01_contratos/'
path_acuerdos = path + '02_acuerdos_confidencialidad'
#===base
base_codificadores = pd.read_excel(path  + '00_Codificadores contratar.xlsx')
base = base_codificadores

#===documentos
acuerdo = path + '01_ACUERDO DE CONFIDENCIALIDAD VF.docx'
contrato = path + '01_CC6193-01 Minuta codificador(a) Icfes VF 231120.docx'


In [ ]:
columns = ['Nombre',
          'Cédula',
          'Correo electrónico']
base = base[columns]
base.head()

base['Cédula'] = base['Cédula'].astype(str)
base['Correo electrónico'].astype(str)

In [30]:
base.head()

,Nombre,Cédula,Correo electrónico
0,Eliana Julieth Calderon Castaneda,1022418108,elianajcalderonc@gmail.com
1,Lady Viviana Abril Barbosa,1026576316,abril.vivi1@gmail.com
2,Byron Sebastian Davila Fandino,1022393019,bsdavilaf@gmail.com
3,Sebastian Ramirez Mosos,1020806611,seramirezmo@unal.edu.co
4,Ana Isabel Castrillon Collazgos,1061726949,anicastrillonco@gmail.com


In [39]:
from time import sleep
#import numpy as np
#base['correo_enviado'] = np.nan

In [ ]:
#Documentos y valores de las funciones
contrato = '01_codificadores/01_CC6193-01 Minuta codificador(a) Icfes VF 231120.docx'
acuerdo = '01_codificadores/01_ACUERDO DE CONFIDENCIALIDAD VF.docx'



#para el envio del correo
content = open('email_codificadores.txt')
email_subject = 'recursoshumanos1@cnccol.com'
email_message = content.read()

from validate_email import validate_email 
list = []
for index, row in base.iterrows():
    correo = row['Correo electrónico']
    is_valid = validate_email(correo)
    if is_valid == False:
        list.append(correo)
    else:
        nombre_completo = row['Nombre']
        #print(nombre_completo)
        cedula = row['Cédula']
        #print(cedula)
        nombre_output = cedula + ' - ' + nombre_completo + ': Contrato como codificador. CNC'
        #Escribiendo contratos
        template = contrato
        path_output_docx = '01_codificadores/01_contratos/'
        path_output_pdf = '01_codificadores/03_attachments/'
        write_docs(cedula, nombre_completo, nombre_output,  template, path_output_docx, path_output_pdf)
        input_file = path_output_pdf + nombre_output + '.pdf'
        #user_pass = cedula
        #owner_pass = 'ContratosCodific4d0r3sCNC!'
        #sleep(3)
        #set_password(input_file, user_pass, owner_pass)

        #Escribiendo acuerdos
        sleep(3)
        template = acuerdo
        nombre_output = cedula + ' - ' + nombre_completo + ': Acuerdo de confidencialidad. CNC'
        path_output_docx = '01_codificadores/02_acuerdos_confidencialidad/'
        path_output_pdf = '01_codificadores/03_attachments/'
        write_docs(cedula, nombre_completo, nombre_output,  template, path_output_docx, path_output_pdf)
        input_file1 = path_output_pdf + nombre_output + '.pdf'
        #user_pass = cedula
        #owner_pass = 'ContratosCodific4d0r3sCNC!'
        #sleep(3)
        #set_password(input_file, user_pass, owner_pass)
        
        sleep(5)
        email_recipient = row['Correo electrónico']
        attachment_location = input_file
        attachment_location1 = input_file1

        send_email(email_recipient, email_subject, email_message, attachment_location, attachment_location1)
        
        

Fields included in 01_codificadores/01_CC6193-01 Minuta codificador(a) Icfes VF 231120.docx: {'CEDULA', 'NOMBRE_COMPLETO', 'VIGENCIA', 'DURACIÓN', 'firma'}
El documento .docx de 1022418108-Eliana Julieth Calderon  Castanedaha sido creado exitosamente.
['/usr/bin/soffice', '--convert-to', 'pdf', '01_codificadores/01_contratos/1022418108 - Eliana Julieth Calderon  Castaneda: Contrato como codificador. CNC.docx']
Fields included in 01_codificadores/01_ACUERDO DE CONFIDENCIALIDAD VF.docx: {'CEDULA', 'NOMBRE_COMPLETO'}
El documento .docx de 1022418108-Eliana Julieth Calderon  Castanedaha sido creado exitosamente.
['/usr/bin/soffice', '--convert-to', 'pdf', '01_codificadores/02_acuerdos_confidencialidad/1022418108 - Eliana Julieth Calderon  Castaneda: Acuerdo de confidencialidad. CNC.docx']
elianajcalderonc@gmail.com
Fields included in 01_codificadores/01_CC6193-01 Minuta codificador(a) Icfes VF 231120.docx: {'CEDULA', 'NOMBRE_COMPLETO', 'VIGENCIA', 'DURACIÓN', 'firma'}
El documento .docx de

Fields included in 01_codificadores/01_ACUERDO DE CONFIDENCIALIDAD VF.docx: {'NOMBRE_COMPLETO', 'CEDULA'}
El documento .docx de 1020738980-Esperanza vuitragoha sido creado exitosamente.
['/usr/bin/soffice', '--convert-to', 'pdf', '01_codificadores/02_acuerdos_confidencialidad1020738980 - Esperanza vuitrago: Contrato como codificador. CNC.docx']
